# Demystifying Compiler Optimizations

This notebook presents an overview of compiler optimizations. We'll discuss what compiler optimizations are, why they're important, and some common types of optimizations. We'll also look at some examples of how these optimizations can be implemented.

## What?
- 🏁 Modifying code (source, bytecode, assembly) in order to increase efficiency by some metric
- 🌲 **AST transformations**

## Why?

- ⏱️ *Minimize execution time*
- 🧠 *Minimize memory usage*
- 🤏 *Minimize code size* (e.g. uglification)

## How?

- 🫣 **Peephole Optimization**
  - *Constant Folding*
  - *Strength Reduction*
- 🔄 **Loop Optimization**
  - *Loop Unrolling*
  - *Loop-Invariant Code Motion*
- 📊 **Data Flow Optimization**
  - *Constant Propagation*
  - *SCCP*
- 🎛️ **Control Flow Optimization**
  - *Dead Code Elimination*
  - *SCCP*
- ⓕ **Function Optimization**
  - *Inline Expansion*
  - *Tail Call Optimization*


## Constant Folding

In [3]:
from ast import *
from typing import Literal
import networkx as nx

cf_code = """
def main():
  x = 5
  y = 1 + 1 + 3
  if x > 0:
    y = 10 * 10
  else:
    y = 20
  return y
"""

In [4]:
class ConstantFolder(NodeTransformer):
    def visit_BinOp(self, node):
        self.generic_visit(node)

        match (node.left, node.op, node.right):
            case (Constant(left), Add(), Constant(right)):
                return Constant(left + right)

            case (Constant(left), Sub(), Constant(right)):
                return Constant(left - right)

            case (Constant(left), Mult(), Constant(right)):
                return Constant(left * right)

            case (Constant(left), Div(), Constant(right)):
                return Constant(left / right)

            case (Constant(left), And(), Constant(right)):
                return Constant(left and right)

            case (Constant(left), Or(), Constant(right)):
                return Constant(left or right)

            case _:
                return node


print(unparse(ConstantFolder().visit(parse(cf_code))))

def main():
    x = 5
    y = 5
    if x > 0:
        y = 100
    else:
        y = 20
    return y


## Strength reduction

In [7]:
sr_code = """
if x % 2 == 0:
  print("even")
else:
  print("odd")
"""

In [8]:
class StrengthReducer(NodeTransformer):
    def visit_BinOp(self, node):
        self.generic_visit(node)

        match (node.left, node.op, node.right):
            case (right, Mod(), Constant(2)):
                return BinOp(right, BitAnd(), Constant(1))

            case (Constant(0), Add(), summand) | (summand, Add(), Constant(0)):
                return summand

            case (Constant(1), Mult(), factor) | (factor, Mult(), Constant(1)):
                return factor

            case (Constant(0), Sub(), subtrahend):
                return UnaryOp(USub(), subtrahend)

            case _:
                return node

print(unparse(StrengthReducer().visit(parse(sr_code))))

if x & 1 == 0:
    print('even')
else:
    print('odd')


## Dead Code Elimination

In [9]:
db_code = """
def main():
  x = 5
  y = 1 + 1 + 3
  if False:
    y = 10 * 10
  else:
    y = 20
  return y
"""

In [10]:
class DeadBranchEliminator(NodeTransformer):
    def visit_If(self, node):
        self.generic_visit(node)
        match node.test:
            case Constant(condition):
                return node.body if condition else node.orelse
            case _:
                return node

print(unparse(DeadBranchEliminator().visit(parse(db_code))))

def main():
    x = 5
    y = 1 + 1 + 3
    y = 20
    return y


In [15]:
dc_code = """
def h():
  g()

def f():
  h()

def g():
  h()

def main():
  h()


main()
"""


In [ ]:
class FunctionEliminator(NodeTransformer):
    def __init__(self, fns: set[str]):
        super().__init__()
        self.fns = fns

    def visit_FunctionDef(self, node):
        return None if node.name in self.fns else self.generic_visit(node)


class DeadFunctionIdentifier(NodeVisitor):
    def __init__(self, draw_graph=True) -> None:
        super().__init__()
        self.graph = nx.DiGraph()
        self.graph.add_node("start")
        self.current_fn: FunctionDef | Literal["start"] = "start"
        self.draw_graph = draw_graph

    def visit_FunctionDef(self, node):
        self.graph.add_node(node.name)
        parent_fn = self.current_fn
        self.current_fn = node
        self.generic_visit(node)
        self.current_fn = parent_fn

    def visit_Call(self, node):
        match (self.current_fn, node.func):
            case (FunctionDef(name=current_fn) | (str() as current_fn), Name(target)):
                self.graph.add_edge(current_fn, target)

        self.generic_visit(node)

    def get_unreachable(self, node):
        self.visit(node)
        reachable = nx.descendants(self.graph, "start") | {"start"}
        unreachable = set(self.graph.nodes) - reachable
        if self.draw_graph:
            pos = nx.spring_layout(self.graph)
            nx.draw(self.graph, pos, with_labels=True, node_color="cyan")
            nx.draw_networkx_nodes(self.graph, pos, nodelist=unreachable, node_color="r")
        return unreachable


class DeadFunctionEliminator:
    def visit(self, node: AST) -> AST:
        unreachable_fns = DeadFunctionIdentifier().get_unreachable(node)
        return FunctionEliminator(unreachable_fns).visit(node)


print(unparse(DeadFunctionEliminator().visit(parse(dc_code))))

# Fin